In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Problem 2

(12p)

**a)** We are observing a time-series $y_t$, $t= 1,2,3, \ldots$ which we model as an AR(p) process observed through noise, that is,
$$
    \begin{cases} \alpha_{t+1} = \sum_{i=1}^{p} a_i \alpha_{t-i} + \eta_r, & \eta_t \sim \mathcal{N}(0, \sigma^2_{\eta}), \\ y_t = b \alpha_t + \varepsilon_t, & \varepsilon_t \sim \mathcal{0,\sigma^2_{\varepsilon}}, \end{cases}
$$
where $a_1, a_2, \ldots, a_p, b, \sigma^2_{\eta}$, and $\sigma^2_{\varepsilon}$ are constants assumed to be known (for now).

To work with this model we need to write it on __state-space form__, write down the full model on state-space form, with a highdimensional hidden AR(1) process. Specify all of the involved matrices and vectors.

<div style="text-align: right"> (2p) </div>

**Solution:**

----

**b)** In the file `data_problem2.csv` the observations from this model is found. From previous experience you set $p = 3$ with $a_1 = 0.55, a_2 = 0.75, a_3 = -0.75, b = 3, \sigma_{\eta}^2 = 0.5^2$, and $\sigma^2_{\varepsilon} = 0.8^2$.

Set the initial distribution to be zero mean and covariance that is an identity matrix with variance 10 for each component.

Using the provided data, provide a plot the estimated values of $\hat{y}_{t | t-1}$ together with $\pm$ 1 standard deviation for each $t = 1,2,3,\ldots$.

Also provide a plot of $\alpha_{t} + \alpha_{t-1} + \alpha_{t-2}$ together with $\pm$ 1 standard deviation for each $t = 1, 2, 3, \ldots$.

_hint: the Kalman filter is available in the `tssltools.py`_
<div style="text-align: right"> (3p) </div>

**Solution:**

(376,)

In [32]:
from tssltools import LGSS, kfs_res

def kalman_filter(y, model: LGSS):
    """Kalman filter for LGSS model with one-dimensional observation.

    :param y: (n,) array of observations. May contain nan, which encodes missing observations.
    :param model: LGSS object with the model specification.
    
    :return kfs_res: Container class with member variables,
        alpha_pred: (d,1,n) array of predicted state means.
        P_pred: (d,d,n) array of predicted state covariances.
        alpha_filt: (d,1,n) array of filtered state means.
        P_filt: (d,d,n) array of filtered state covariances.
        y_pred: (n,) array of means of p(y_t | y_{1:t-1})
        F_pred: (n,) array of variances of p(y_t | y_{1:t-1})
    """

    n = len(y)
    d = model.d
    alpha_pred = np.zeros((d, 1, n))
    P_pred = np.zeros((d, d, n))
    alpha_filt = np.zeros((d, 1, n))
    P_filt = np.zeros((d, d, n))
    y_pred = np.zeros(n)
    F_pred = np.zeros(n)

    T, R, Q, Z, H, a1, P1 = model.get_params() 

    for t in range(n):
        # Time update (predict)
        if t == 0:  
            alpha_pred[:, 0, 0] = a1 
            P_pred[:, :, 0] = P1
        else:
            alpha_pred[:, :, t] = T @ alpha_filt[:,:,t-1]
            P_pred[:, :, t] = T @ P_filt[:,:,t-1] @ T.T + R@Q@R.T


        # Compute prediction of current output
        y_pred[t] = Z @ alpha_pred[:, :, t]
        F_pred[t] = Z @ P_pred[:, :, t] @ Z.T + H

        # Measurement update
        if np.isnan(y[t]):
            alpha_filt[:, :, t] = alpha_pred[:, :, t].copy()
            P_filt[:, :, t] = P_pred[:, :, t].copy()
        else:
            K = P_pred[:, :, t] @ Z.T / F_pred[t]
            print((K * (y[t] - y_pred[t])).shape)
            alpha_filt[:, :, t] = alpha_pred[:, :, t] + K * (y[t] - y_pred[t])
            P_filt[:, :, t] = (np.identity(d) - K @ Z) @ P_pred[:, :, t]

    kf = kfs_res(alpha_pred, P_pred, alpha_filt, P_filt, y_pred, F_pred)
    return kf

----

**c)** You are abit unsure about the values of $\sigma_{\eta}^2$ and $\sigma_{\varepsilon}^2$ and decide to retrain these parameters again. You remember that the EM-algorithm can be used for this problem, **describe the EM-algorithm briefly and find the updating formulas for the two variances**. To your help you have the complete-data log-likelihood,
$$
    \log p_{\theta}(\alpha_{1:n},y_{1:n}) = \text{const.} - \frac{1}{2}\sum_{i=1}^{n}\left[ \log \sigma_{\varepsilon}^2 + \log \sigma_{\eta}^2 + \varepsilon_{i}^{\top} \sigma_{\varepsilon}^{-2} \varepsilon_{i} + \eta_i^{\top} \sigma_{\eta}^{-2} \eta_i \right],
$$
where $\varepsilon_{i} = y_i - Z \alpha_i$ and $\eta_i = R^{\top} (\alpha_i - T \alpha_{i-1})$.
<div style="text-align: right"> (4p) </div>

**Solution:**

---

**d)** Use the results from **c)** to implement the EM-algorithm and provide plots of the parameter trajectories for the two variances.

_hint: if you failed with **c)** you can use the function `theta_update_wrong` from `tssltools.py`. The Kalmans smoother is available in `tssltools.py`_
<div style="text-align: right"> (3p) </div>

**Solution:**